In [2]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [86]:
import matplotlib
matplotlib.use('TKAgg')

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
import pandas as pd

In [87]:
import datetime

Import all the necessary libraries for matplotlib and pandas

In [88]:
df=pd.read_csv("C:\\Users\\danil\\Downloads\\yellow_tripdata_2019-01.csv")

read the first dataset for yellow taxi as csv with pandas

In [89]:
df=df.drop(columns=['VendorID', 'passenger_count','store_and_fwd_flag','store_and_fwd_flag','extra','extra','tip_amount','tolls_amount','improvement_surcharge','total_amount','congestion_surcharge','mta_tax','payment_type'])


Drop all unnecessary columns. Most of them are the money paid and information about the driver that has no impact on our assigment

In [90]:
df=df[df.trip_distance != 0.00]

Drop all the rows that has 0 trip distance because the rows dont contribute much without distance

In [91]:
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
df=df.drop(df[df.tpep_pickup_datetime > df.tpep_dropoff_datetime].index)
df=df.drop(df[df.PULocationID>263].index)
df=df.drop(df[df.DOLocationID>263].index)

Make the pick up and drop off date to pandas dtype date so that we can use it later when compared. Drop the rows with greater pick up time then drop of time and drop rows that has greater pick up location id and drop of location id 

In [64]:
fh=pd.read_csv("C:\\Users\\danil\\Downloads\\fhv_tripdata_2019-01.csv")


Read the for hire taxi dataset to add to the other one

In [65]:
fh=fh.dropna()

Drop all the rows with NaN data

In [66]:
fh=fh.drop(columns=['dispatching_base_num','SR_Flag'])

Drop unnecessary columns

In [67]:
fh.pickup_datetime = pd.to_datetime(fh.pickup_datetime)
fh.dropoff_datetime = pd.to_datetime(fh.dropoff_datetime)
fh=fh.drop(fh[fh.PULocationID>263].index)
fh=fh.drop(fh[fh.DOLocationID>263].index)
fh=fh.drop(fh[fh.pickup_datetime > fh.dropoff_datetime].index)

same as yellow Make the pick up and drop off date to pandas dtype date and drop some rows

In [92]:
gt=pd.read_csv("C:\\Users\\danil\\Downloads\\green_tripdata_2019-01.csv")

Read green taxi dataset clean and add to the yellow

In [93]:
gt=gt.drop(columns=['VendorID', 'passenger_count','store_and_fwd_flag','store_and_fwd_flag','extra','extra','tip_amount','tolls_amount','improvement_surcharge','total_amount','congestion_surcharge','mta_tax','payment_type','ehail_fee','trip_type'])


Has the same columns as yellow so we do the same cleaning process

In [94]:
gt.lpep_pickup_datetime = pd.to_datetime(gt.lpep_pickup_datetime)
gt.lpep_dropoff_datetime = pd.to_datetime(gt.lpep_dropoff_datetime)

In [95]:
gt=gt[gt.trip_distance != 0.00]

In [96]:
gt=gt.drop(gt[gt.lpep_pickup_datetime > gt.lpep_dropoff_datetime].index)
gt=gt.drop(gt[gt.PULocationID>263].index)
gt=gt.drop(gt[gt.DOLocationID>263].index)

In [97]:
df['pickup'] = (df['tpep_pickup_datetime']-df['tpep_pickup_datetime'].dt.normalize()).dt.total_seconds()
df['time'] = (df.tpep_dropoff_datetime-df.tpep_pickup_datetime).dt.total_seconds()
df=df.drop(columns=['tpep_dropoff_datetime'])
df=df.drop(columns=['tpep_pickup_datetime'])

Make a new column 'pickup' that has the pickup time in seconds without the date, midnight starts at 0. So we can use it in our model later

Make a new column 'time' that has the driving time in seconds and starts at 0 just when the client is picked up. Used in the model aswell 

After we get the columns time and pick up drop unnecessary column

In [74]:
fh['time'] = (fh.dropoff_datetime-fh.pickup_datetime).dt.total_seconds()
fh=fh.drop(columns=['dropoff_datetime'])
fh['pickup'] = (fh['pickup_datetime']-fh['pickup_datetime'].dt.normalize()).dt.total_seconds()
fh=fh.drop(columns=['pickup_datetime'])

Do the same with for hire dataset

In [98]:
gt['time'] = (gt.lpep_dropoff_datetime-gt.lpep_pickup_datetime).dt.total_seconds()
gt=gt.drop(columns=['lpep_dropoff_datetime'])
gt['pickup'] = (gt['lpep_pickup_datetime']-gt['lpep_pickup_datetime'].dt.normalize()).dt.total_seconds()
gt=gt.drop(columns=['lpep_pickup_datetime'])

Do the same with the green dataset

In [99]:
df=df.append(gt)

C:\Users\danil\AppData\Local\Temp\ipykernel_6688\3538197625.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(gt)


green and yellow have the same columns so append green into yellow to get 1 dataset

In [100]:
df=df.drop(df[df.time==0].index)

some more cleaning, remove all the wrongly input rows like time traveled 0, fare amount under 2.5 because starting fee is 2.5 and rides that take longer then 8 hours 

In [101]:
df=df.drop(df[df.fare_amount<=2.5].index)

In [102]:
df=df.drop(df[df.time>60*60*8].index)

In [74]:
df.to_csv('yellow_taxi.csv')

make the new files so we dont need to run code again

In [75]:
fh.to_csv('for_hire.csv')

In [76]:
gt.to_csv('green_taxi.csv')

In [103]:
X = df[['trip_distance','pickup']]
Y = df['time']

  
X = X.values.reshape(len(X), 2)
Y = Y.values.reshape(len(Y), 1)


Give X and Y values to use in the models and set the size. We use trip distance and pickup time for X and Y is the time travled  

In [104]:
df.describe()

,trip_distance,RatecodeID,PULocationID,DOLocationID,fare_amount,pickup,time
count,7.999598e+06,7.999598e+06,7.999598e+06,7.999598e+06,7.999598e+06,7.999598e+06,7.999598e+06
mean,2.852626e+00,1.063802e+00,1.593659e+02,1.591539e+02,1.230683e+01,5.143937e+04,7.866176e+02
std,3.683954e+00,5.533856e-01,6.760539e+01,7.063965e+01,2.206185e+02,2.129533e+04,6.369239e+02
min,1.000000e-02,1.000000e+00,1.000000e+00,1.000000e+00,2.800000e+00,0.000000e+00,1.000000e+00
25%,9.400000e-01,1.000000e+00,1.130000e+02,1.070000e+02,6.000000e+00,3.631000e+04,3.730000e+02
50%,1.590000e+00,1.000000e+00,1.610000e+02,1.610000e+02,9.000000e+00,5.377000e+04,6.160000e+02
75%,2.900000e+00,1.000000e+00,2.310000e+02,2.330000e+02,1.350000e+01,6.856900e+04,1.004000e+03
max,8.318000e+02,9.900000e+01,2.630000e+02,2.630000e+02,6.232599e+05,8.639900e+04,2.877300e+04


Use this code to check for bad data

In [105]:
X_train = X[:int(0.8*len(X))]
X_test = X[int(0.8*len(X)):]

Y_train = Y[:int(0.8*len(Y))]
Y_test = Y[int(0.8*len(Y)):]


divide into test sets and train sets for the model

In [106]:
from sklearn.metrics import mean_squared_error, r2_score


Add the score measures

In [107]:
from sklearn.tree import DecisionTreeRegressor

t = DecisionTreeRegressor()
t = t.fit(X_train, Y_train)
Y_hat = t.predict(X_test)
r2_score(Y_test, Y_hat)

0.44537176548862356

test the decision tree first and we got 0.44

In [108]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()
lm = lm.fit(X_train, Y_train)
Y_hat = lm.predict(X_test)
r2_score(Y_test, Y_hat)


0.5607402507346347

Test linear model and got a better result then the descision tree at 0.56

In [ ]:
from sklearn.ensemble import RandomForestRegressor

f = RandomForestRegressor()
f = t.fit(X_train, Y_train)
Y_hat = t.predict(X_test)
r2_score(Y_test, Y_hat)

Lastly test the random forrest and got the best result but the time it takes to calculate the random tree is very very long, so we decided to use the linear model

In [ ]:
from joblib import dump, load
dump(lm, 'linnearYellow.joblib') 

Use joblib with to copy the linnear model into out assigment